**Домашнее задание**

Имеются данные adult.csv (см. в материалах к занятию).  
Целевой переменной является уровень дохода income (крайний правый столбец).  
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html  
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.  
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.  
Готовый ноутбук выложить на гитхаб и прислать ссылку.  

In [1]:
import pandas as pd
from IPython.display import display, HTML
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [2]:
adult = pd.read_csv("adult.csv", na_values="?")
adult.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
print("1. Подготавливаем данные.")
print(adult.info())
print("\nВ данных есть пропущенные значения. Попробуем их отбросить.")
adult.dropna(inplace=True)
print(adult.info())
print("\nПотеряли около 3,5К строк, будем надеяться, что это не так критично для датасета в 49К строк.")

1. Подготавливаем данные.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
None

В данных есть пропущенные значения. Попробуе

In [4]:
print("Оценим визуально мусор в категориальных признаках.")
categories = ["workclass", "education", "marital-status", "occupation", "relationship", "race",
              "gender", "native-country", "income"]
for cat in categories:
    print(adult[cat].unique())
del categories[categories.index("income")]
print("\nПохоже, все нормально.")

Оценим визуально мусор в категориальных признаках.
['Private' 'Local-gov' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay']
['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' '5th-6th' 'Assoc-voc' '9th' 'Doctorate'
 '12th' '1st-4th' 'Preschool']
['Never-married' 'Married-civ-spouse' 'Widowed' 'Separated' 'Divorced'
 'Married-spouse-absent' 'Married-AF-spouse']
['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' 'Other-service'
 'Prof-specialty' 'Craft-repair' 'Adm-clerical' 'Exec-managerial'
 'Tech-support' 'Sales' 'Priv-house-serv' 'Transport-moving'
 'Handlers-cleaners' 'Armed-Forces']
['Own-child' 'Husband' 'Not-in-family' 'Unmarried' 'Wife' 'Other-relative']
['Black' 'White' 'Other' 'Amer-Indian-Eskimo' 'Asian-Pac-Islander']
['Male' 'Female']
['United-States' 'Peru' 'Guatemala' 'Mexico' 'Dominican-Republic'
 'Ireland' 'Germany' 'Philippines' 'Thailand' 'Haiti' 'El-Salvador'
 'Puerto-Rico' 'Vietnam' 'Sou

In [5]:
print("Формируем набор исходных данных.")
X = pd.get_dummies(adult, columns=categories)
del X["income"]
X.head()

Формируем набор исходных данных.


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,0,0,30,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
print("Формируем набор результата.")
income_en = LabelEncoder()
income_en.fit(adult["income"])
print(f"Нумеруемые энкодером значения: {income_en.classes_}.")
Y = pd.Series(income_en.transform(adult["income"]))
Y.head()

Формируем набор результата.
Нумеруемые энкодером значения: ['<=50K' '>50K'].


0    0
1    0
2    1
3    1
4    0
dtype: int64

In [7]:
print("Делим данные на выборки.")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

Делим данные на выборки.


In [8]:
print("2. Обучаем модель логистической регрессии на тренировочных данных.")
model_log = make_pipeline(StandardScaler(), LogisticRegression())
model_log.fit(X_train, Y_train);

2. Обучаем модель логистической регрессии на тренировочных данных.


In [9]:
def score_model(model, income_en, X_data, Y_data):
    columns = [["prediction", "prediction"], income_en.classes_]
    index = [["data", "data"], income_en.classes_]
    Y_predict = model.predict(X_data)
    cmtx = pd.DataFrame(confusion_matrix(Y_data, Y_predict), columns=columns, index=index)
    print("\nМатрица ошибок")
    display(HTML(cmtx.to_html()))
    accur = model.score(X_data, Y_data)
    print(f"\nКачество предсказаний: {accur}.")

print("Оцениваем предсказания модели на тренировочных данных.")
score_model(model_log, income_en, X_train, Y_train)

print("\nОцениваем предсказания модели на тестовых данных.")
score_model(model_log, income_en, X_test, Y_test)

Оцениваем предсказания модели на тренировочных данных.

Матрица ошибок



Качество предсказаний: 0.848017690728163.

Оцениваем предсказания модели на тестовых данных.

Матрица ошибок



Качество предсказаний: 0.8500036854131348.


In [10]:
print("3. Обучаем модель SVM.")
model_svm = make_pipeline(StandardScaler(), SVC())
model_svm.fit(X_train, Y_train);

3. Обучаем модель SVM.


In [11]:
print("Оцениваем предсказания модели на тренировочных данных.")
score_model(model_svm, income_en, X_train, Y_train)

print("\nОцениваем предсказания модели на тестовых данных.")
score_model(model_svm, income_en, X_test, Y_test)

Оцениваем предсказания модели на тренировочных данных.

Матрица ошибок



Качество предсказаний: 0.8584109935239299.

Оцениваем предсказания модели на тестовых данных.

Матрица ошибок



Качество предсказаний: 0.8510356010908823.


In [12]:
print("Вывод: у обеих моделей качество предсказаний примерно одинаковое.")

Вывод: у обеих моделей качество предсказаний примерно одинаковое.
